In [17]:
import os
import torch
import torch.nn as nn
import numpy as np
import torchvision
import torch.nn.functional as F
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import torch.optim as optim
from PIL import Image
import cv2


In [18]:
train_tfms = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
#                                  transforms.RandomRotation(20),
#                                  transforms.RandomAffine(15),
                                ])

valid_tfms = transforms.Compose([transforms.Grayscale(num_output_channels=1),
#                                  transforms.Resize((48,48)),
                                transforms.ToTensor(),
                                ])

In [19]:
image_path = "data_old/train/"
train_dataset = ImageFolder(image_path, transform = train_tfms)
image_path_test = "data_old/test/"
test_dataset = ImageFolder(image_path_test, transform = valid_tfms)

In [20]:
train_loader = torch.utils.data.DataLoader(train_dataset, 64, 
                                             shuffle=True, num_workers = 2)
val_loader = torch.utils.data.DataLoader(test_dataset, 1, shuffle = False, )

In [21]:
def show_batch(dl):
    for images,lables in dl:
        plt.imshow(make_grid(images[:64], nrow=8).permute(1, 2, 0))
        plt.show()
        break

In [22]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

device = get_default_device()
print(device)

cuda


In [23]:
# class CNN(nn.Module):
#     def __init__(self, drop=0.5):
#         super().__init__()

#         self.conv0a = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
#         self.conv0b = nn.Conv2d(32, out_channels=32, kernel_size=3, padding=1)

#         self.conv1a = nn.Conv2d(32, 64, 3, padding=1)
#         self.conv1b = nn.Conv2d(64, 64, 3, padding=1)
        
#         self.conv2a = nn.Conv2d(64, 128, 3, padding=1)
#         self.conv2b = nn.Conv2d(128, 128, 3, padding=1)

#         self.conv3a = nn.Conv2d(128, 256, 3, padding=1)
#         self.conv3b = nn.Conv2d(256, 256, 3, padding=1)

#         self.conv4a = nn.Conv2d(256, 512, 3, padding=1)
#         self.conv4b = nn.Conv2d(512, 512, 3, padding=1)

#         self.pool = nn.MaxPool2d(kernel_size=2,)
        
#         self.bn0a = nn.BatchNorm2d(32)
#         self.bn0b = nn.BatchNorm2d(32)
        
#         self.bn1a = nn.BatchNorm2d(64)
#         self.bn1b = nn.BatchNorm2d(64)

#         self.bn2a = nn.BatchNorm2d(128)
#         self.bn2b = nn.BatchNorm2d(128)

#         self.bn3a = nn.BatchNorm2d(256)
#         self.bn3b = nn.BatchNorm2d(256)

#         self.bn4a = nn.BatchNorm2d(512)
#         self.bn4b = nn.BatchNorm2d(512)

#         self.lin1 = nn.Linear(512, 6)

#         self.drop = nn.Dropout(p=drop)

#     def forward(self, x):
#         x = F.relu(self.bn0a(self.conv0a(x)),inplace = True)
#         x = F.relu(self.bn0b(self.conv0b(x)),inplace = True)
#         x = self.pool(x)
        
#         x = F.relu(self.bn1a(self.conv1a(x)),inplace = True)
#         x = F.relu(self.bn1b(self.conv1b(x)),inplace = True)
#         x = self.pool(x)

#         x = F.relu(self.bn2a(self.conv2a(x)),inplace = True)
#         x = F.relu(self.bn2b(self.conv2b(x)),inplace = True)
#         x = self.pool(x)

#         x = F.relu(self.bn3a(self.conv3a(x)),inplace = True)
#         x = F.relu(self.bn3b(self.conv3b(x)),inplace = True)
#         x = self.pool(x)

#         x = F.relu(self.bn4a(self.conv4a(x)),inplace = True)
#         x = F.relu(self.bn4b(self.conv4b(x)),inplace = True)
#         x = self.pool(x)

#         x = x.view(-1, 512)
#         x = self.lin1(x)

#         return x

In [24]:
# class CNN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.cnn1 = nn.Sequential(nn.Conv2d(1,32,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace=True),
#                                   nn.BatchNorm2d(32),
#                                   nn.Conv2d(32,64,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(64),
#                                   nn.MaxPool2d((2,2)),
#                                   nn.Dropout(0.5),
#                                  )
#         self.cnn2 = nn.Sequential(nn.Conv2d(64,128,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(128),
#                                   nn.Dropout(0.2),
#                                   nn.Conv2d(128,128,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(128),
#                                   nn.Dropout(0.2),
#                                   nn.Conv2d(128,128,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(128),
#                                   nn.MaxPool2d((2,2)),
#                                   nn.Dropout(0.5),  
#                                  )
#         self.cnn3 = nn.Sequential(nn.Conv2d(128,256,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(256),
#                                   nn.Dropout(0.2),
#                                   nn.Conv2d(256,256,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(256),
#                                   nn.Dropout(0.2),
#                                   nn.MaxPool2d((2,2)),
#                                   nn.Conv2d(256,256,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(256),
#                                   nn.Dropout(0.2),
#                                   nn.Conv2d(256,256,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(256),
#                                   nn.MaxPool2d((2,2)),
#                                   nn.Dropout(0.5),   
#                                 )
#         self.cnn4 = nn.Sequential(nn.Conv2d(256,512,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(512),
#                                   nn.Dropout(0.2),
#                                   nn.Conv2d(512,512,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(512),
#                                   nn.Dropout(0.2),
#                                   nn.Conv2d(512,512,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(512),
#                                   nn.Dropout(0.2),
#                                   nn.Conv2d(512,512,kernel_size = 3,padding = 1),
#                                   nn.ReLU(inplace = True),
#                                   nn.BatchNorm2d(512),
#                                   nn.MaxPool2d((2,2)),
#                                   nn.Dropout(0.5), 
#                                  )        
#         self.flatten = nn.Sequential(nn.Linear(512,7),
#                                     )
        
#     def forward(self,x):
#         x = self.cnn1(x)
#         x = self.cnn2(x)
#         x = self.cnn3(x)
#         x = self.cnn4(x)
# #         x = self.cnn5(x)
#         x = x.view(x.size(0), -1)
#         x = self.flatten(x)
#         return x

In [32]:
class CNN3(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn1 = nn.Sequential(nn.Conv2d(1, 32, 3, padding = 1),
                                  nn.ReLU(inplace = True),
                                  nn.BatchNorm2d(32),
                                  nn.Conv2d(32, 64, 3, padding = 1),
                                  nn.ReLU(inplace = True),
                                  nn.BatchNorm2d(64),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.4),
                                 )
        self.cnn2 = nn.Sequential(nn.Conv2d(64, 128, 3, padding = 1),
                                  nn.ReLU(inplace = True),
                                  nn.BatchNorm2d(128),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.4),
                                  nn.Conv2d(128, 128, 3, padding = 1),
                                  nn.ReLU(inplace = True),
                                  nn.BatchNorm2d(128),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.4),
                                  nn.Conv2d(128, 256, 3, padding = 1),
                                  nn.ReLU(inplace = True),
                                  nn.BatchNorm2d(256),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.4),
                                  nn.Conv2d(256, 256, 3, padding = 1),
                                  nn.ReLU(inplace = True),
                                  nn.BatchNorm2d(256),
                                  nn.MaxPool2d(2),
                                  nn.Dropout(0.5),
                                 )
        self.fc = nn.Sequential(nn.Linear(256,1024),
                                nn.ReLU(inplace = True),
                                nn.Dropout(0.35),
                                nn.Linear(1024,6),
                                )
        
    def forward(self,x):
        x = self.cnn1(x)
        x = self.cnn2(x)
#         print(x.shape)
        x = x.view(x.size(0),-1)
        x = self.fc(x)
        return x

In [36]:
net = CNN3()
net = net.cuda()

In [40]:
# net.load_state_dict(torch.load("Emotion_detection_weights_SGD(0.001,0.92).pth"))

In [37]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.001,)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.75, patience=1, verbose=True)

In [38]:
def train(model,dataloader,validloader,criterion,optimizer,epochs=50,scheduler=scheduler):
    max_valid_acc = 0
    train_acc,val_acc = 0,0
    for e in range(epochs):
        train_loss = 0.0
        model.train()     # Optional when not using Model Specific layer
        for data, labels in dataloader:
#             print(labels.shape[0])
            data, labels = data.to(device), labels.to(device)
#             print(data.shape)
            optimizer.zero_grad()
            target = model(data)
#             print(target.shape)
            loss = criterion(target.float(),labels.long())
            loss.backward()
            optimizer.step()
            train_loss = loss.item() * data.size(0)
            train_acc += torch.sum((torch.max(target, 1)[1] == labels.data),0)
            
        valid_loss = 0.0
        model.eval()     # Optional when not using Model Specific layer
        for data, labels in validloader:
            
            data, labels = data.to(device), labels.to(device)

            target = model(data)
            loss = criterion(target.float(),labels.long())
            valid_loss = loss.item() * data.size(0)
            val_acc += torch.sum((torch.max(target, 1)[1] == labels.data),0)
            
            
        print(f'Epoch {e+1} \t\t Training Loss: {train_loss / len(train_dataset)} \t\t Validation Loss: {valid_loss / len(validloader)}')
        print("Validation Accuracy ... :",val_acc/(len(test_dataset)))
        print("Train Accuracy ... :",train_acc/23659)
        
        if val_acc > max_valid_acc:
            print(f'Validation Acc Increased({max_valid_acc:.6f}--->{val_acc:.6f}) \t Saving The Model')
            max_valid_acc = val_acc
            # Saving State Dict
            torch.save(model.state_dict(), 'HIGH_Adam.pth')
#         scheduler.step(val_acc)
        train_acc = 0
        val_acc = 0
    return model

In [39]:
net.to('cuda')
train(net,train_loader,val_loader,criterion,optimizer,100)
print("Done!!")

Epoch 1 		 Training Loss: 0.0028866859975487293 		 Validation Loss: 4.1358364096570824e-05
Validation Accuracy ... : tensor(0.4164, device='cuda:0')
Train Accuracy ... : tensor(0.3082, device='cuda:0')
Validation Acc Increased(0.000000--->2937.000000) 	 Saving The Model
Epoch 2 		 Training Loss: 0.002612962819109059 		 Validation Loss: 1.7610083704009624e-05
Validation Accuracy ... : tensor(0.4984, device='cuda:0')
Train Accuracy ... : tensor(0.4521, device='cuda:0')
Validation Acc Increased(2937.000000--->3515.000000) 	 Saving The Model
Epoch 3 		 Training Loss: 0.002096429718865289 		 Validation Loss: 1.731942549472529e-05
Validation Accuracy ... : tensor(0.5368, device='cuda:0')
Train Accuracy ... : tensor(0.5081, device='cuda:0')
Validation Acc Increased(3515.000000--->3786.000000) 	 Saving The Model
Epoch 4 		 Training Loss: 0.0022650604537039092 		 Validation Loss: 2.0881217117260892e-05
Validation Accuracy ... : tensor(0.5700, device='cuda:0')
Train Accuracy ... : tensor(0.5400,

Epoch 36 		 Training Loss: 0.0018052745876890241 		 Validation Loss: 7.85315676300336e-07
Validation Accuracy ... : tensor(0.6518, device='cuda:0')
Train Accuracy ... : tensor(0.6835, device='cuda:0')
Validation Acc Increased(4556.000000--->4597.000000) 	 Saving The Model
Epoch 37 		 Training Loss: 0.0019081155314589991 		 Validation Loss: 1.6743516108169296e-06
Validation Accuracy ... : tensor(0.6444, device='cuda:0')
Train Accuracy ... : tensor(0.6852, device='cuda:0')
Epoch 38 		 Training Loss: 0.0015464847111942791 		 Validation Loss: 2.1258958278776617e-06
Validation Accuracy ... : tensor(0.6508, device='cuda:0')
Train Accuracy ... : tensor(0.6883, device='cuda:0')
Epoch 39 		 Training Loss: 0.0016531032648953523 		 Validation Loss: 1.5923136196355523e-06
Validation Accuracy ... : tensor(0.6519, device='cuda:0')
Train Accuracy ... : tensor(0.6853, device='cuda:0')
Validation Acc Increased(4597.000000--->4598.000000) 	 Saving The Model
Epoch 40 		 Training Loss: 0.00144183583211417

Epoch 74 		 Training Loss: 0.0015569740594035448 		 Validation Loss: 4.445497904173046e-07
Validation Accuracy ... : tensor(0.6614, device='cuda:0')
Train Accuracy ... : tensor(0.7277, device='cuda:0')
Epoch 75 		 Training Loss: 0.0014687307531183417 		 Validation Loss: 6.429260544078529e-07
Validation Accuracy ... : tensor(0.6645, device='cuda:0')
Train Accuracy ... : tensor(0.7260, device='cuda:0')
Epoch 76 		 Training Loss: 0.0014657006889882713 		 Validation Loss: 2.1356394915598159e-07
Validation Accuracy ... : tensor(0.6660, device='cuda:0')
Train Accuracy ... : tensor(0.7304, device='cuda:0')
Validation Acc Increased(4687.000000--->4697.000000) 	 Saving The Model
Epoch 77 		 Training Loss: 0.001629329912590258 		 Validation Loss: 5.646661060229609e-07
Validation Accuracy ... : tensor(0.6675, device='cuda:0')
Train Accuracy ... : tensor(0.7345, device='cuda:0')
Validation Acc Increased(4697.000000--->4708.000000) 	 Saving The Model
Epoch 78 		 Training Loss: 0.002005191465820929 

In [41]:
train(net,train_loader,val_loader,criterion,optimizer,100)
print("Done!!")

Epoch 1 		 Training Loss: 0.0012170790084684738 		 Validation Loss: 6.978778331240119e-08
Validation Accuracy ... : tensor(0.6695, device='cuda:0')
Train Accuracy ... : tensor(0.7461, device='cuda:0')
Validation Acc Increased(0.000000--->4722.000000) 	 Saving The Model
Epoch 2 		 Training Loss: 0.0015358123538470028 		 Validation Loss: 1.0107129383882394e-07
Validation Accuracy ... : tensor(0.6708, device='cuda:0')
Train Accuracy ... : tensor(0.7531, device='cuda:0')
Validation Acc Increased(4722.000000--->4731.000000) 	 Saving The Model
Epoch 3 		 Training Loss: 0.0013221303525597157 		 Validation Loss: 2.6998768102255644e-07
Validation Accuracy ... : tensor(0.6716, device='cuda:0')
Train Accuracy ... : tensor(0.7456, device='cuda:0')
Validation Acc Increased(4731.000000--->4737.000000) 	 Saving The Model
Epoch 4 		 Training Loss: 0.0013032469556789206 		 Validation Loss: 5.29409743138751e-07
Validation Accuracy ... : tensor(0.6657, device='cuda:0')
Train Accuracy ... : tensor(0.7499,

Epoch 40 		 Training Loss: 0.0009525303262652773 		 Validation Loss: 1.0242247413941584e-07
Validation Accuracy ... : tensor(0.6633, device='cuda:0')
Train Accuracy ... : tensor(0.7749, device='cuda:0')
Epoch 41 		 Training Loss: 0.0016332716989998866 		 Validation Loss: 2.7105047219753364e-07
Validation Accuracy ... : tensor(0.6715, device='cuda:0')
Train Accuracy ... : tensor(0.7737, device='cuda:0')
Epoch 42 		 Training Loss: 0.0012206463572954891 		 Validation Loss: 3.717096495248909e-07
Validation Accuracy ... : tensor(0.6769, device='cuda:0')
Train Accuracy ... : tensor(0.7742, device='cuda:0')
Validation Acc Increased(4755.000000--->4774.000000) 	 Saving The Model
Epoch 43 		 Training Loss: 0.001219446052204479 		 Validation Loss: 3.2028441491548547e-07
Validation Accuracy ... : tensor(0.6644, device='cuda:0')
Train Accuracy ... : tensor(0.7734, device='cuda:0')
Epoch 44 		 Training Loss: 0.001365677332637286 		 Validation Loss: 1.559865617522209e-07
Validation Accuracy ... : te

Epoch 80 		 Training Loss: 0.0014674928934887202 		 Validation Loss: 1.1797705821045388e-07
Validation Accuracy ... : tensor(0.6705, device='cuda:0')
Train Accuracy ... : tensor(0.7910, device='cuda:0')
Epoch 81 		 Training Loss: 0.0010409854445794617 		 Validation Loss: 6.598666542447764e-08
Validation Accuracy ... : tensor(0.6779, device='cuda:0')
Train Accuracy ... : tensor(0.7907, device='cuda:0')
Epoch 82 		 Training Loss: 0.0012564356110312722 		 Validation Loss: 1.8108986789780814e-07
Validation Accuracy ... : tensor(0.6735, device='cuda:0')
Train Accuracy ... : tensor(0.7927, device='cuda:0')
Epoch 83 		 Training Loss: 0.0012960630715495409 		 Validation Loss: 2.7774758008411044e-07
Validation Accuracy ... : tensor(0.6730, device='cuda:0')
Train Accuracy ... : tensor(0.7950, device='cuda:0')
Epoch 84 		 Training Loss: 0.0011561386272160694 		 Validation Loss: 1.1708199885178251e-07
Validation Accuracy ... : tensor(0.6745, device='cuda:0')
Train Accuracy ... : tensor(0.7962, dev

In [16]:
import cv2
from PIL import Image
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
net.eval()
cap = cv2.VideoCapture(0)
emotion_dict = {0: "Angry", 1: "Fearful", 2: "Happy", 3: "Neutral", 4: "Sad", 5: "Surprised"}
while True:
    
    _, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)
    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x-10, y-50), (x+w+20, y+h+50), (255, 0, 0), 2)
        roi_gray_frame = img[y:y + h, x:x + w]
        PILimage = Image.fromarray(roi_gray_frame)
        inputs = valid_tfms(PILimage)
        inputs = inputs.unsqueeze(0)
        emotion_prediction = net(inputs)
        maxindex = int(torch.argmax(emotion_prediction))
        cv2.putText(img, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

    cv2.imshow('img', img)
    k = cv2.waitKey(30) & 0xff
    if k==27:
        break
cap.release()
cv2.destroyAllWindows()